# フレア検出 学習ノート 03: 回転周期と星黒点解析

このノートでは、`rotation_period` と `flux_diff` を中心に、
回転周期推定と星黒点面積・比率の推定方法を学びます。


## このノートで学ぶこと（予定）

- Lomb–Scargle 法の概要と `ROTATION_FREQUENCY_GRID` の意味
- パワースペクトルから回転周期と誤差を読み取る方法
- 振幅から星黒点面積・比率を推定する式の直感的な理解
- 複数恒星の回転周期とフレア活動の関係をざっくり眺める方法


## 1. 概要と前提

このノートでは、EK Dra を題材にして

- `rotation_period()` がどのように回転周期を推定しているか
- `flux_diff()` から何が計算されているか

を確認し、そのうえで DS Tuc A / V889 Her を軽く比較します。

前提として：

- `data/TESS/EK_Dra/`, `DS_Tuc_A`, `V889_Her` に FITS が配置されている
- `src/base_flare_detector.py` / `src/flarepy_*.py` が import 可能
- 01・02 ノートで前処理とフレア検出の流れを一度眺めている

ことを想定しています。


In [ ]:
"""EK Dra の 1 ファイルを process_data=True で処理し、回転周期と星黒点を調べる。"""

import sys
from pathlib import Path

import numpy as np
import pandas as pd
import plotly.express as px

from src.flarepy_EK_Dra import FlareDetector_EK_Dra
from src.base_flare_detector import ROTATION_FREQUENCY_GRID, _ROTATION_PERIODS

NOTEBOOK_DIR = Path().resolve()
PROJECT_ROOT = NOTEBOOK_DIR.parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

DATA_EK = PROJECT_ROOT / "data" / "TESS" / "EK_Dra"
print("DATA_EK:", DATA_EK, "exists:", DATA_EK.exists())

fits_files_ek = sorted(DATA_EK.glob("*.fits")) if DATA_EK.exists() else []
print("found", len(fits_files_ek), "EK Dra FITS files")

example_ek = fits_files_ek[0] if fits_files_ek else None
print("example_ek:", example_ek)

if example_ek is None:
    detector_ek = None
    print("EK Dra の FITS が見つからないため、このノートの残りはスキップしてください。")
else:
    detector_ek = FlareDetector_EK_Dra(
        file=str(example_ek),
        process_data=True,
        ene_thres_low=5e33,
        ene_thres_high=2e40,
    )
    print("rotation period per [day]:", detector_ek.per)
    print("rotation period error per_err [day]:", detector_ek.per_err)
    print("brightness_variation_amplitude:", detector_ek.brightness_variation_amplitude)

## 2. Lomb–Scargle 法による回転周期推定

`rotation_period()` は、`ROTATION_FREQUENCY_GRID` 上で
Lomb–Scargle 期間解析を行い、パワーが最大になる周波数から
回転周期 `per` を決めています。

ここでは、実際に計算されたパワースペクトルを

- 横軸: 周期 [day]
- 縦軸: パワー

でプロットし、ピークがどこにあるか確認します。


In [ ]:
if detector_ek is None or detector_ek.power is None:
    print("detector_ek が未定義か、rotation_period がまだ計算されていません。")
else:
    periods = _ROTATION_PERIODS
    power = detector_ek.power

    print("len(periods):", len(periods))
    print("len(power):", len(power))

    fig = px.line(
        x=periods,
        y=power,
        labels={"x": "Period [day]", "y": "Lomb–Scargle power"},
        title="03-1: Lomb–Scargle power vs period (EK Dra)",
    )
    # 周期の短い側が右に来るように反転
    fig.update_xaxes(autorange="reversed")

    # 推定された回転周期に縦線を引く
    fig.add_vline(
        x=detector_ek.per,
        line_color="red",
        line_dash="dash",
        annotation_text="per",
        annotation_position="top",
    )

    fig.show()

## 3. パワースペクトルの可視化と周期誤差

上の図では、パワーが最大になる周期 `per` に赤い縦線を引いています。

`rotation_period()` では、この周辺で

- パワーが最大値の 1/2 を超える周波数帯を探し
- その帯の幅から周期の不確かさ `per_err` を見積もっています。

詳細な実装は `src/base_flare_detector.py` を参照してくださいが、
ノートブックとしては

- 「メインピークの幅が狭いほど周期推定がシャープ」
- 「幅が広い場合は `per_err` も大きくなる」

という直感を持っておくと、図の読み方が理解しやすくなります。


## 4. `flux_diff` による星黒点面積・比率の推定

`flux_diff()` は、光度曲線の振幅から

- 星黒点の実効面積 `starspot`
- 光度変動の比率としての `starspot_ratio`

を推定します。

内部ではざっくりと：

1. 正規化光度 `mPDCSAPflux` の分布から、
   - 下位数％と上位数％を使って振幅（max-min）を計算
2. 恒星半径と温度、温度コントラストから
   - 「この振幅がどれくらいの黒点面積に対応するか」を計算

という流れになっています。

ここでは、EK Dra に対して計算された値を
そのまま表示してみます。


In [ ]:
if detector_ek is None:
    print("detector_ek が未定義です。先頭のセルを実行してください。")
else:
    # 明示的に flux_diff を呼び直しても良い（process_data 内でも呼ばれている）
    detector_ek.flux_diff()

    print("brightness_variation_amplitude:", detector_ek.brightness_variation_amplitude)
    print("starspot (area-like quantity):", detector_ek.starspot)
    print("starspot_ratio:", detector_ek.starspot_ratio)

## 5. 複数恒星の比較例

最後に、EK Dra / DS Tuc A / V889 Her について

- 回転周期 `per`
- 光度変動振幅 `brightness_variation_amplitude`
- 簡易なフレア発生率 `flare_ratio = flare_number / precise_obs_time`

を 1 ファイルずつ比較してみます。

ここでは、各恒星について先頭の FITS ファイルだけを処理し、
軽量な比較サンプルとして扱います。


In [ ]:
from src.flarepy_DS_Tuc_A import FlareDetector_DS_Tuc_A
from src.flarepy_V889_Her import FlareDetector_V889_Her

DATA_DS = PROJECT_ROOT / "data" / "TESS" / "DS_Tuc_A"
DATA_V889 = PROJECT_ROOT / "data" / "TESS" / "V889_Her"

print("DATA_DS:", DATA_DS, "exists:", DATA_DS.exists())
print("DATA_V889:", DATA_V889, "exists:", DATA_V889.exists())

star_rows = []

# EK Dra (すでに detector_ek がある想定)
if detector_ek is not None:
    flare_ratio_ek = (
        detector_ek.flare_number / detector_ek.precise_obs_time
        if detector_ek.precise_obs_time > 0
        else np.nan
    )
    star_rows.append(
        {
            "star": "EK_Dra",
            "per": detector_ek.per,
            "per_err": detector_ek.per_err,
            "amp": detector_ek.brightness_variation_amplitude,
            "flare_ratio": flare_ratio_ek,
        }
    )

# DS Tuc A
if DATA_DS.exists():
    ds_files = sorted(DATA_DS.glob("*.fits"))
    if ds_files:
        ds_example = ds_files[0]
        ds_det = FlareDetector_DS_Tuc_A(
            file=str(ds_example),
            process_data=True,
            ene_thres_low=5e33,
            ene_thres_high=2e40,
        )
        flare_ratio_ds = (
            ds_det.flare_number / ds_det.precise_obs_time
            if ds_det.precise_obs_time > 0
            else np.nan
        )
        star_rows.append(
            {
                "star": "DS_Tuc_A",
                "per": ds_det.per,
                "per_err": ds_det.per_err,
                "amp": ds_det.brightness_variation_amplitude,
                "flare_ratio": flare_ratio_ds,
            }
        )

# V889 Her
if DATA_V889.exists():
    v889_files = sorted(DATA_V889.glob("*.fits"))
    if v889_files:
        v889_example = v889_files[0]
        v889_det = FlareDetector_V889_Her(
            file=str(v889_example),
            process_data=True,
            ene_thres_low=5e33,
            ene_thres_high=2e40,
        )
        flare_ratio_v889 = (
            v889_det.flare_number / v889_det.precise_obs_time
            if v889_det.precise_obs_time > 0
            else np.nan
        )
        star_rows.append(
            {
                "star": "V889_Her",
                "per": v889_det.per,
                "per_err": v889_det.per_err,
                "amp": v889_det.brightness_variation_amplitude,
                "flare_ratio": flare_ratio_v889,
            }
        )

if not star_rows:
    print("どの恒星についてもデータが見つからなかったため、このセルはスキップします。")
else:
    df_star = pd.DataFrame(star_rows)
    df_star

In [ ]:
if "df_star" not in globals():
    print("先に上のセルを実行して df_star を作成してください。")
else:
    fig_scatter = px.scatter(
        df_star,
        x="per",
        y="flare_ratio",
        text="star",
        labels={"per": "Rotation period [day]", "flare_ratio": "Flare rate [1/day]"},
        title="05: Rotation period vs flare rate (per file)",
    )
    fig_scatter.update_traces(textposition="top center")
    fig_scatter.show()